In [1]:
# ติดตั้งบน Notebook ถ้ายังไม่ได้ติดตั้ง
!pip install pandas numpy scikit-learn openpyxl

import os
import ast
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, classification_report
from lightgbm import LGBMClassifier


NER

In [3]:


from openai import OpenAI

# Initialize the client with your API key and the OpenTyphoon base URL

prompt = f"""
กรุณาวิเคราะห์ข้อความภาษาไทยต่อไปนี้อย่างละเอียด และระบุชื่อบุคคลทั้งหมด รวมทั้งชื่อสั้นๆ ด้วย

แทนที่ทุกชื่อบุคคลด้วยรูปแบบ {{people_1}}, {{people_2}}, ... ตามลำดับการปรากฏในข้อความ 
- ตรวจสอบชื่อให้ละเอียด แม้จะเป็นชื่อสั้นๆ เช่น 'แดง', 'ดำ' ก็ต้องแทนที่
- อย่าลืมตรวจสอบชื่อที่อาจมีคำนำหน้า เช่น 'นาย', 'นาง', 'นางสาว', 'คุณ', หรือไม่มีคำนำหน้าก็ได้
- หากมีชื่อจริงตามด้วยนามสกุล ให้นับเป็นบุคคลเดียวกัน เช่น 'นายแดง ใจดี' ให้เป็น '{{people_1}}'

ส่งคืนเฉพาะข้อความที่แปลงแล้วเท่านั้น ไม่ต้องมีคำอธิบายหรือข้อความอื่นใดนอกจากข้อความที่ได้แทนที่ชื่อบุคคลแล้ว

ลักษณะของการแยกชื่อบุคคล คือ [ชื่อบุคคล] + กิริยา หรือ กิริยา + [ชื่อบุคคล] เช่น จุฬาภร + ดำเนิน
"""
client = OpenAI(
    api_key="sk-75Gt45VtcvieHJihgyYjFlrc9SHVph5sGMruGCHoffF8bFKc",
    base_url="https://api.opentyphoon.ai/v1"
)

# Make a completion request

text = "เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิต"
response = client.chat.completions.create(
    model="typhoon-v2.1-12b-instruct",
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"ข้อความ: {text}"}
    ],
    max_tokens=512,
    temperature=0.0
)

# Print the response
print(response.choices[0].message.content)

{people_1} แพ้หมากรุกเลยชักปืนขึ้นมายิง{people_2}จนเสียชีวิต


In [4]:
import ast
import pandas as pd
import json
from ollama import Client

client = Client()

def extract_named_entities(text):
    """
    สกัดชื่อบุคคลจากข้อความภาษาไทยและแทนที่ด้วยรูปแบบมาตรฐาน
    
    Args:
        text (str): ข้อความภาษาไทยที่ต้องการประมวลผล
    
    Returns:
        str: ข้อความเดิมที่มีการแทนที่ชื่อบุคคลด้วย {people_1}, {people_2}, ...
    """
    prompt = f"""
กรุณาวิเคราะห์ข้อความภาษาไทยต่อไปนี้อย่างละเอียด และระบุชื่อบุคคลทั้งหมด รวมทั้งชื่อสั้นๆ ด้วย

แทนที่ทุกชื่อบุคคลด้วยรูปแบบ {{people_1}}, {{people_2}}, ... ตามลำดับการปรากฏในข้อความ 
- ตรวจสอบชื่อให้ละเอียด แม้จะเป็นชื่อสั้นๆ เช่น 'แดง', 'ดำ' ก็ต้องแทนที่
- อย่าลืมตรวจสอบชื่อที่อาจมีคำนำหน้า เช่น 'นาย', 'นาง', 'นางสาว', 'คุณ', หรือไม่มีคำนำหน้าก็ได้
- หากมีชื่อจริงตามด้วยนามสกุล ให้นับเป็นบุคคลเดียวกัน เช่น 'นายแดง ใจดี' ให้เป็น '{{people_1}}' หรือ "แดง"

ส่งคืนเฉพาะข้อความที่แปลงแล้วเท่านั้น ไม่ต้องมีคำอธิบายหรือข้อความอื่นใดนอกจากข้อความที่ได้แทนที่ชื่อบุคคลแล้ว

ลักษณะของการแยกชื่อบุคคล คือ 
- [ชื่อบุคคล] + กิริยา
- กิริยา + [ชื่อบุคคล]
- [ชื่อบุคคล] + คำวิเศษณ์ 
เช่น [จุฬาภร] + ดำเนิน / [บัวขาว] + ชนะ

ข้อความ: {text}
"""
    
    response = client.chat(
        model='hf.co/scb10x/typhoon2.1-gemma3-12b-gguf:Q4_K_M',
        messages=[{'role': 'user', 'content': prompt}],
    )
    
    content = response['message']['content'].strip()
    
    # Clean up any explanations that might be included
    lines = content.split('\n')
    for line in lines:
        if len(line) > 0 and not line.startswith("ข้อความ:") and not line.startswith("-"):
            return line.strip()
    
    return content

# ตัวอย่างการใช้งาน
sample_texts = [
    "เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิต"
    "วิภาวดีนั่งคิดในการวางแผนฆ่าสุชาดา แต่สุชาดาชิงฆ่าก่อน"
]

for text in sample_texts:
    result = extract_named_entities(text)
    print(f"\nInput: {text}")
    print(f"Output: {result}")



Input: เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิตวิภาวดีนั่งคิดในการวางแผนฆ่าสุชาดา แต่สุชาดาชิงฆ่าก่อน
Output: เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิง{people_1}จนเสียชีวิต{people_2}นั่งคิดในการวางแผนฆ่า{people_3}แต่{people_3}ชิงฆ่าก่อน


In [5]:
sample_texts = [
    "วิภาวดีนั่งคิดในการวางแผนฆ่าสุชาดา แต่สุชาดาชิงฆ่าก่อน"
]

for text in sample_texts:
    result = extract_named_entities(text)
    print(f"\nInput: {text}")
    print(f"Output: {result}")


Input: วิภาวดีนั่งคิดในการวางแผนฆ่าสุชาดา แต่สุชาดาชิงฆ่าก่อน
Output: {people_1} นั่งคิดในการวางแผนฆ่า {people_2} แต่ {people_2} ชิงฆ่าก่อน


เรื่มโมเดล

In [6]:
# 1) โหลดไฟล์ train_half.csv และ dev_half.csv
import pandas as pd, ast
train_df = pd.read_csv(r"C:\miewmiew\Hackathon\Hack2\crime-charges-analysis\train_half.csv")
dev_df   = pd.read_csv(r"C:\miewmiew\Hackathon\Hack2\crime-charges-analysis\dev_half.csv")

# 2) ฟังก์ชันช่วย parse คอลัมน์ answers
def parse_answers(cell: str) -> dict[str, list[int]]:
    raw = ast.literal_eval(cell)
    out = {}
    for d in raw:
        for person, cs in d.items():
            out[person] = ast.literal_eval(cs)
    return out

# 3) สร้างคอลัมน์ parsed
train_df["parsed"] = train_df["answers"].apply(parse_answers)
dev_df  ["parsed"] = dev_df  ["answers"].apply(parse_answers)


In [7]:
# สร้าง X_train, y_train, X_dev, y_dev
X_train, y_train = [], []
for _, row in train_df.iterrows():
    for person, charges in row["parsed"].items():
        X_train.append(f"{person} [SEP] {row['story']}")
        y_train.append(charges)

X_dev, y_dev = [], []
for _, row in dev_df.iterrows():
    for person, charges in row["parsed"].items():
        X_dev.append(f"{person} [SEP] {row['story']}")
        y_dev.append(charges)

# สร้าง binary labels
from sklearn.preprocessing import MultiLabelBinarizer
mlb     = MultiLabelBinarizer(classes=[1,2,3,4,5,6,7])
Y_train = mlb.fit_transform(y_train)
Y_dev   = mlb.transform(y_dev)


In [8]:
# ตรวจ shape ว่าถูกต้อง
print("X_train:", len(X_train), "samples")
print("Y_train shape:", Y_train.shape)
print("X_dev  :", len(X_dev),   "samples")
print("Y_dev   shape:", Y_dev.shape)

X_train: 1398 samples
Y_train shape: (1398, 7)
X_dev  : 1392 samples
Y_dev   shape: (1392, 7)


ML baseline

In [9]:
# ======== เซลล์ถัดไป: ML Baseline =========
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

# 1) สร้าง pipeline
ml_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), max_features=20000)),
    ("clf",   OneVsRestClassifier(
        LogisticRegression(solver="liblinear", class_weight="balanced"),
        n_jobs=-1
    ))
])

# 2) เทรนบน X_train, Y_train
ml_pipeline.fit(X_train, Y_train)

# 3) ทำนายบน X_dev
Y_pred = ml_pipeline.predict(X_dev)

# 4) ประเมินผล
print("ML Baseline F1 (macro):", f1_score(Y_dev, Y_pred, average="macro"))
print("\nClassification Report:")
print(classification_report(
    Y_dev, Y_pred,
    target_names=[f"Class {c}" for c in mlb.classes_]
))


ML Baseline F1 (macro): 0.6605556701269235

Classification Report:
              precision    recall  f1-score   support

     Class 1       0.46      0.56      0.51       307
     Class 2       0.65      0.79      0.71       417
     Class 3       0.63      0.71      0.66       190
     Class 4       0.66      0.83      0.74       255
     Class 5       0.55      0.75      0.63       373
     Class 6       0.64      0.69      0.66       251
     Class 7       0.59      0.89      0.71       247

   micro avg       0.59      0.75      0.66      2040
   macro avg       0.60      0.75      0.66      2040
weighted avg       0.59      0.75      0.66      2040
 samples avg       0.57      0.72      0.61      2040



c:\miewmiew\Hackathon\Hack2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
# ======== เซลล์ถัดไป: Threshold Tuning =========
import numpy as np
from sklearn.metrics import f1_score

# 1) รับ probability จาก ML pipeline บน X_dev
probs = ml_pipeline.predict_proba(X_dev)  # shape = (n_samples, n_classes)

# 2) หา threshold ที่ให้ F1 ดีที่สุดสำหรับแต่ละคลาส
classes_list = [1,2,3,4,5,6,7]
best_thresh = {}

for i, cls in enumerate(classes_list):
    best_f1, best_t = 0.0, 0.5
    # ลอง threshold ตั้งแต่ 0.1 ถึง 0.9
    for t in np.linspace(0.1, 0.9, 17):
        y_t = (probs[:, i] >= t).astype(int)
        f1 = f1_score(Y_dev[:, i], y_t)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    best_thresh[cls] = best_t
    print(f"Class {cls}: best threshold = {best_t:.2f}, F1 = {best_f1:.3f}")

print("\nThresholds dictionary:", best_thresh)


Class 1: best threshold = 0.40, F1 = 0.524
Class 2: best threshold = 0.50, F1 = 0.712
Class 3: best threshold = 0.45, F1 = 0.676
Class 4: best threshold = 0.55, F1 = 0.739
Class 5: best threshold = 0.50, F1 = 0.635
Class 6: best threshold = 0.45, F1 = 0.677
Class 7: best threshold = 0.50, F1 = 0.706

Thresholds dictionary: {1: np.float64(0.4), 2: np.float64(0.5), 3: np.float64(0.45000000000000007), 4: np.float64(0.55), 5: np.float64(0.5), 6: np.float64(0.45000000000000007), 7: np.float64(0.5)}


In [11]:
# ======== เซลล์นิยาม rule-based classifier =========
import re

# Lexicon เบื้องต้น (ขยายเพิ่มได้)
crime_keywords = {
    2: ["ขโมย","ลัก","ชิงทรัพย์","ยักยอก","แอบขโมย"],
    3: ["ประมาท","ไม่ตั้งใจ","ละเลย","ชะล่าใจ","เผลอ"],
    4: ["บุกรุก","เข้ามาโดยมิชอบ","ล้ำเขต","แอบเข้า"],
    5: ["เจตนา","ตั้งใจ","จงใจ","มุ่งประสงค์","ไตร่ตรอง","วางแผน"],
    6: ["ฉ้อโกง","ปลอมแปลง","หลอกลวง","แอบอ้าง","ปลอมลายเซ็น"],
    7: ["ฆ่า","สังหาร","ทำให้เสียชีวิต","วางยาพิษ","ยิงตาย"]
}

def extract_characters_regex(text: str) -> list[str]:
    """
    หาชื่อโดยง่าย ด้วย regex จับกลุ่มคำไทยยาว ≥2 ตัว
    (ในระยะเริ่มต้นก่อนใช้ NER)
    """
    tokens = re.findall(r"[ก-๙]{2,}", text)
    return list(set(tokens))

def classify_text(text: str) -> dict[str, list[int]]:
    """
    Rule-based classifier: คืน dict person→[list of charges]
    """
    names = extract_characters_regex(text)
    result = {n: [] for n in names}
    # แยกเป็นประโยค
    for sent in re.split(r"[\.!?]", text):
        sl = sent.lower()
        # ตรวจ keyword แต่ละคลาส
        detected = {cls for cls, kws in crime_keywords.items()
                    if any(kw in sl for kw in kws)}
        # เงื่อนไขพิเศษ คลาส 7 ต้องรวม 5 หรือ 3
        if 7 in detected:
            if 5 in detected: detected.add(5)
            elif 3 in detected: detected.add(3)
        # map ไปยังคนที่ปรากฏในประโยค
        for n in names:
            if n in sent:
                result[n].extend(detected)
    # คืนค่า sorted unique หรือ [1] ถ้าไม่เจอ
    return {n: sorted(set(v)) or [1] for n, v in result.items()}

# ทดสอบว่า classify_text พร้อมใช้งาน
print(classify_text(dev_df["story"].iloc[0]))


{'คร่าชีวิตเกศวสาที่นอนหลับอยู่ใต้ตึกที่กำลังก่อสร้าง': [1], 'ทำให้เกิดแผ่นดินไหว': [1], 'รอยเลื่อนสะกา': [1]}


In [ ]:
# ======== เซลล์ Hybrid Evaluation (Corrected) =========
import numpy as np
from sklearn.metrics import f1_score, classification_report

classes_list = [1,2,3,4,5,6,7]

# 1) สร้าง lists ของ true_mat และ pred_mat
true_rows, pred_rows = [], []

for _, row in dev_df.iterrows():
    gt = parse_answers(row["answers"])        # dict person→true charges
    hy = classify_hybrid(row["story"])        # dict person→pred charges
    for person, true_charges in gt.items():
        # แถวจริง
        true_row = [1 if cls in true_charges else 0 for cls in classes_list]
        true_rows.append(true_row)
        # แถวทำนาย
        pred_charges = hy.get(person, [1])
        pred_row = [1 if cls in pred_charges else 0 for cls in classes_list]
        pred_rows.append(pred_row)

# 2) แปลงเป็น NumPy arrays
true_mat = np.array(true_rows)
pred_mat = np.array(pred_rows)

# 3) คำนวณ F1 macro
print("Hybrid F1 (macro):", f1_score(true_mat, pred_mat, average="macro"))

# 4) แสดง classification report ต่อคลาส
print("\nClassification Report:")
print(classification_report(
    true_mat, pred_mat,
    target_names=[f"Class {c}" for c in classes_list]
))

#ติด error

NameError: name 'classify_hybrid' is not defined

In [ ]:
# ======== เซลล์ Hybrid Evaluation (Wide‐format) =========
import numpy as np
from sklearn.metrics import f1_score, classification_report

classes_list = [1,2,3,4,5,6,7]

# 1) สร้าง true_rows และ pred_rows เป็น list ของ list (n_samples × 7)
true_rows, pred_rows = [], []

for _, row in dev_df.iterrows():
    gt   = parse_answers(row["answers"])       # dict person→true charges
    hy   = classify_hybrid(row["story"])       # dict person→pred charges
    for person, true_charges in gt.items():
        # แถวจริง
        true_row = [1 if cls in true_charges else 0 for cls in classes_list]
        # แถบทำนาย
        pred_charges = hy.get(person, [1])
        pred_row = [1 if cls in pred_charges else 0 for cls in classes_list]
        
        true_rows.append(true_row)
        pred_rows.append(pred_row)

# 2) แปลงเป็น NumPy arrays
true_mat = np.array(true_rows)
pred_mat = np.array(pred_rows)

# 3) คำนวณ F1‐macro
print("Hybrid F1 (macro):", f1_score(true_mat, pred_mat, average="macro"))

# 4) แสดง classification report ต่อแต่ละคลาส
print("\nClassification Report:")
print(classification_report(
    true_mat, pred_mat,
    target_names=[f"Class {c}" for c in classes_list]
))


Hybrid F1 (macro): 0.06225202810493542

Classification Report:
              precision    recall  f1-score   support

     Class 1       0.22      0.99      0.36       307
     Class 2       0.50      0.00      0.00       417
     Class 3       0.33      0.01      0.01       190
     Class 4       0.67      0.01      0.02       255
     Class 5       0.00      0.00      0.00       373
     Class 6       0.46      0.02      0.05       251
     Class 7       0.00      0.00      0.00       247

   micro avg       0.22      0.15      0.18      2040
   macro avg       0.31      0.15      0.06      2040
weighted avg       0.31      0.15      0.06      2040
 samples avg       0.22      0.22      0.22      2040



c:\miewmiew\Hackathon\Hack2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
